https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [1]:
# necessary imports
import os, json
import pandas as pd
import numpy as np
import glob
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('german')
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()
import gensim
from gensim import corpora, models
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read lda_tweets
df = pd.read_csv('../../data/BTW17_Twitter/lda/lda_tweets.csv')
df.drop(columns='Unnamed: 0', axis=1, inplace=True)
df.head()

,created_at,id,text,user,entities,tags
0,2017-06-28,8.798635e+17,RT @Junge_Freiheit: AfD-Vize Beatrix von Storc...,"{'id': 728326774027919360, 'id_str': '72832677...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
1,2017-06-28,8.798879e+17,RT @AfD_Bund: Dr. @Alice_Weidel:\n»#Ehefuerall...,"{'id': 1222045254, 'id_str': '1222045254', 'na...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
2,2017-06-28,8.798901e+17,"RT @hessenSPD: Abstimmen, abstimmen, abstimmen...","{'id': 2443895924, 'id_str': '2443895924', 'na...","{'hashtags': [{'text': 'ehefueralle', 'indices...",ehefueralle
3,2017-06-28,8.798903e+17,RT @Die_Gruenen: Hallo @spdbt! Jetzt hindert E...,"{'id': 12302322, 'id_str': '12302322', 'name':...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle
4,2017-06-28,8.798903e+17,"RT @Die_Gruenen: ""Wer die Debatte auch nur hal...","{'id': 391983270, 'id_str': '391983270', 'name...","{'hashtags': [{'text': 'Ehefueralle', 'indices...",ehefueralle


In [3]:
# preprocessing function
def preprocess(df):
    # remove mentions, links, non-alphanumeric
    df['text'] = df['text'].apply(lambda x: re.sub(r'(@[\w]+)|(\w+:\/\/\S+)', '', x))
    # tokenizer
    df['tokens'] = df['text'].apply(lambda x: regexp_tokenize(x.lower(), "[\w]+"))
    # stopword removal
    df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in stopwords])
    # lemmatizer
    df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(item) for item in x])
    # remove short words
    df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if len(word) >= 3])
    # tbd: only nouns and adjectives

In [4]:
preprocess(df)

In [5]:
print('original document: ')
words = []
for word in df['text'][5].split(' '):
    words.append(word)
print(words)
print('\n\ntokenized and lemmatized document: ')
print(df['tokens'][5])

original document: 
['RT', ':', 'Wir', 'werden', 'die', '#Ehefueralle', 'in', 'dieser', 'Woche', 'im', 'Bundestag', 'zur', 'Abstimmung', 'stellen.', 'Endlich', '-', 'denn', 'Liebe', 'kennt', 'kein', 'Ge…', '']


tokenized and lemmatized document: 
['ehefueralle', 'woche', 'bundestag', 'abstimmung', 'stellen', 'endlich', 'liebe', 'kennt']


In [6]:
hashtag_list = df['tags'].unique().tolist()

for index in tqdm(range(len(hashtag_list))):
    hashtag = hashtag_list[index]
    print(hashtag)
    
    # filter df
    temp_df = df[df['tags']==hashtag]
    
    # get dictionary
    dictionary = gensim.corpora.Dictionary(temp_df['tokens'])
    #dictionary.filter_extremes(no_above=0.5, keep_n=100000)
        
    # get bag of words corpus
    bow_corpus = [dictionary.doc2bow(doc) for doc in temp_df['tokens']]
    # lda_model
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=1, id2word=dictionary, passes=2, workers=2)
    
    # print topics bow
    for idx, topic in lda_model.print_topics(-1):
        print(f'Topic BOW: {hashtag} \nWords: {topic}')    
    
    #ifidf
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=1, id2word=dictionary, passes=2, workers=4)
            
    # print topics tfidf
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print(f'Topic TF-IDF: {hashtag} \nWords: {topic}\n')

ehefueralle
Topic BOW: ehefueralle 
Words: 0.119*"ehefueralle" + 0.013*"cdu" + 0.013*"ehe" + 0.012*"merkel" + 0.011*"homo" + 0.011*"abstimmung" + 0.011*"freitag" + 0.011*"marx" + 0.011*"wegen" + 0.011*"rechtsausschuss"
Topic TF-IDF: ehefueralle 
Words: 0.009*"homo" + 0.009*"ehe" + 0.009*"marx" + 0.009*"wegen" + 0.009*"kardinal" + 0.009*"antwortet" + 0.009*"gespannt" + 0.009*"brief" + 0.009*"offener" + 0.009*"rechtsausschuss"

gesundheit
Topic BOW: gesundheit 
Words: 0.112*"gesundheit" + 0.045*"pflege" + 0.040*"umkämpft" + 0.040*"neues" + 0.040*"dossier" + 0.040*"prekär" + 0.031*"macht" + 0.031*"wer" + 0.031*"verkehrswende" + 0.031*"seehofer"
Topic TF-IDF: gesundheit 
Words: 0.045*"dossier" + 0.045*"prekär" + 0.045*"umkämpft" + 0.045*"neues" + 0.043*"pflege" + 0.032*"menschliche" + 0.032*"verbieten" + 0.032*"bund" + 0.032*"tiermast" + 0.032*"weiterhin"

afd
Topic BOW: afd 
Words: 0.127*"afd" + 0.065*"traudichdeutschland" + 0.030*"btw17" + 0.011*"http" + 0.010*"merkel" + 0.009*"deutschla

Topic BOW: dresden 
Words: 0.118*"dresden" + 0.084*"vorfällen" + 0.084*"leider" + 0.084*"traudichdeutschland" + 0.084*"samstag" + 0.084*"gewalttätigen" + 0.084*"mehreren" + 0.084*"afd" + 0.024*"menschen" + 0.014*"mitsicherheitgutankommen"
Topic TF-IDF: dresden 
Words: 0.091*"traudichdeutschland" + 0.091*"afd" + 0.091*"leider" + 0.091*"mehreren" + 0.091*"vorfällen" + 0.091*"samstag" + 0.091*"gewalttätigen" + 0.020*"menschen" + 0.015*"forderungen" + 0.015*"u18"

grueneversenken
Topic BOW: grueneversenken 
Words: 0.112*"grueneversenken" + 0.047*"traudichdeutschland" + 0.036*"btw17" + 0.023*"grüne" + 0.023*"afd" + 0.015*"afdwaehlen" + 0.014*"familiennachzug" + 0.013*"verschleierungsverbot" + 0.013*"spitzenkandidatin" + 0.010*"btw"
Topic TF-IDF: grueneversenken 
Words: 0.023*"traudichdeutschland" + 0.016*"grüne" + 0.016*"verschleierungsverbot" + 0.016*"spitzenkandidatin" + 0.015*"familiennachzug" + 0.014*"btw17" + 0.013*"afd" + 0.013*"afdwaehlen" + 0.008*"btw2017" + 0.008*"btw"

pflege
Topi

Topic BOW: grün 
Words: 0.095*"grün" + 0.043*"grundeinkommen" + 0.043*"zweitstimme" + 0.043*"wahlaufruf" + 0.026*"bitte" + 0.026*"grüner" + 0.026*"unterstützung" + 0.026*"unterstützen" + 0.022*"kandidierenden" + 0.022*"http"
Topic TF-IDF: grün 
Words: 0.028*"zweitstimme" + 0.028*"wahlaufruf" + 0.028*"grundeinkommen" + 0.028*"grüner" + 0.028*"unterstützen" + 0.028*"bitte" + 0.028*"unterstützung" + 0.024*"grünen" + 0.024*"http" + 0.024*"kandidierenden"

sed
Topic BOW: sed 
Words: 0.090*"sed" + 0.034*"klare" + 0.034*"desaster" + 0.034*"wesentlicher" + 0.034*"versäumte" + 0.034*"nachahmer" + 0.034*"grund" + 0.034*"fürs" + 0.034*"btw2017" + 0.034*"absage"
Topic TF-IDF: sed 
Words: 0.036*"danke" + 0.036*"gespräch" + 0.036*"mauerschützenpartei" + 0.036*"fürs" + 0.036*"propagandablatt" + 0.032*"versäumte" + 0.032*"btw2017" + 0.032*"desaster" + 0.032*"nachahmer" + 0.032*"klare"

gez
Topic BOW: gez 
Words: 0.196*"gez" + 0.128*"schande" + 0.042*"afd" + 0.030*"zdf" + 0.030*"landes" + 0.030*"gewähl

Topic TF-IDF: merkel 
Words: 0.013*"islam" + 0.011*"gesehen" + 0.011*"rückgängigmachen" + 0.011*"methode" + 0.011*"atomausstieg" + 0.011*"schief" + 0.011*"beim" + 0.011*"gehen" + 0.011*"mal" + 0.010*"afd"

schulzzug
Topic BOW: schulzzug 
Words: 0.159*"schulzzug" + 0.143*"entgleist" + 0.135*"völlig" + 0.135*"siegessicherheit" + 0.135*"unterstellungen" + 0.135*"gejammer" + 0.016*"nirwana" + 0.016*"verzweiflung" + 0.016*"fährt" + 0.016*"afd"
Topic TF-IDF: schulzzug 
Words: 0.152*"völlig" + 0.152*"gejammer" + 0.152*"siegessicherheit" + 0.152*"unterstellungen" + 0.122*"entgleist" + 0.026*"afd" + 0.026*"nirwana" + 0.026*"fährt" + 0.026*"wählen" + 0.025*"asozialdemokratischen"

türkei
Topic BOW: türkei 
Words: 0.208*"reisewarnung" + 0.116*"türkei" + 0.096*"bundesregierung" + 0.095*"absurdistan" + 0.095*"erdogan" + 0.095*"diktator" + 0.095*"drückt" + 0.017*"mehr" + 0.017*"deutschland" + 0.015*"repression"
Topic TF-IDF: türkei 
Words: 0.136*"erdogan" + 0.136*"drückt" + 0.136*"absurdistan" + 0.1

Topic TF-IDF: csu 
Words: 0.047*"http" + 0.046*"stoppen" + 0.046*"nimmt" + 0.046*"beitrittsverhandlungen" + 0.046*"weitere" + 0.046*"deutsche" + 0.046*"geiseln" + 0.045*"erdogan" + 0.043*"warum" + 0.043*"spd"

schulz
Topic BOW: schulz 
Words: 0.098*"schulz" + 0.067*"tvduell" + 0.050*"merkel" + 0.038*"afd" + 0.022*"btw17" + 0.019*"traudichdeutschla" + 0.016*"thesen" + 0.016*"moderatoren" + 0.016*"attackieren" + 0.016*"ecke"
Topic TF-IDF: schulz 
Words: 0.018*"afd" + 0.018*"ecke" + 0.018*"thesen" + 0.018*"vier" + 0.018*"attackieren" + 0.018*"moderatoren" + 0.018*"interessant" + 0.016*"merkel" + 0.015*"traudichdeutschla" + 0.015*"btw17"

zdf
Topic BOW: zdf 
Words: 0.104*"zdf" + 0.084*"schlagabtausch" + 0.034*"müssen" + 0.031*"rentenreform" + 0.031*"politiker" + 0.031*"einzahlen" + 0.031*"große" + 0.031*"brauchen" + 0.026*"tot" + 0.026*"polizei"
Topic TF-IDF: zdf 
Words: 0.032*"einzahlen" + 0.032*"politiker" + 0.032*"rentenreform" + 0.032*"brauchen" + 0.032*"große" + 0.031*"müssen" + 0.025

Topic BOW: köln 
Words: 0.094*"köln" + 0.074*"afd" + 0.074*"btw17" + 0.074*"traudichdeutschland" + 0.060*"wahlkampf" + 0.047*"geht" + 0.047*"puste" + 0.047*"land" + 0.047*"mülheim" + 0.020*"ergebnis"
Topic TF-IDF: köln 
Words: 0.057*"puste" + 0.057*"mülheim" + 0.057*"geht" + 0.057*"land" + 0.047*"wahlkampf" + 0.036*"blaue" + 0.036*"allee" + 0.034*"btw17" + 0.034*"afd" + 0.034*"traudichdeutschland"

terror
Topic BOW: terror 
Words: 0.085*"terror" + 0.050*"spd" + 0.046*"barcelona" + 0.035*"berlin" + 0.035*"btw17" + 0.031*"spitzenkandidatin" + 0.031*"traudichdeutschland" + 0.031*"skandal" + 0.031*"egal" + 0.031*"afd"
Topic TF-IDF: terror 
Words: 0.028*"spitzenkandidatin" + 0.028*"högl" + 0.028*"zeigt" + 0.028*"skandal" + 0.028*"egal" + 0.028*"eva" + 0.027*"berlin" + 0.027*"btw17" + 0.023*"barcelona" + 0.023*"spd"

digitalisierung
Topic BOW: digitalisierung 
Words: 0.101*"digitalisierung" + 0.039*"beim" + 0.030*"thema" + 0.026*"5kampf" + 0.023*"geht" + 0.021*"kompetenz" + 0.021*"schulen" +

Topic TF-IDF: ard 
Words: 0.117*"btw17" + 0.098*"löscht" + 0.098*"news" + 0.098*"seiten" + 0.098*"traudichdeutschland" + 0.098*"fake" + 0.098*"facebook" + 0.098*"gibt" + 0.098*"zdf" + 0.011*"cdu"

fürsieda
Topic BOW: fürsieda 
Words: 0.090*"fürsieda" + 0.077*"bundestag" + 0.077*"cdu" + 0.077*"steineke" + 0.051*"fedidwgugl" + 0.051*"tage" + 0.038*"morgen" + 0.038*"btw17" + 0.038*"tour" + 0.038*"gilt"
Topic TF-IDF: fürsieda 
Words: 0.058*"gilt" + 0.058*"morgen" + 0.050*"fedidwgugl" + 0.043*"btw17" + 0.042*"region" + 0.042*"bilanz" + 0.039*"ikarus" + 0.039*"tour" + 0.038*"kitas" + 0.038*"gründe"

klima
Topic BOW: klima 
Words: 0.077*"klima" + 0.037*"nix" + 0.033*"wählbar" + 0.033*"schützt" + 0.033*"politik" + 0.033*"menschen" + 0.033*"darumgrün" + 0.033*"natur" + 0.033*"september" + 0.033*"tiere"
Topic TF-IDF: klima 
Words: 0.033*"menschen" + 0.033*"politik" + 0.033*"wählbar" + 0.033*"tiere" + 0.033*"september" + 0.033*"darumgrün" + 0.033*"schützt" + 0.033*"natur" + 0.028*"nix" + 0.018*"a

Topic BOW: polizei 
Words: 0.125*"amp" + 0.067*"polizei" + 0.063*"danke" + 0.063*"ausstattung" + 0.063*"bessere" + 0.062*"kopf" + 0.062*"haltet" + 0.062*"mio" + 0.062*"überstunden" + 0.062*"personal"
Topic TF-IDF: polizei 
Words: 0.097*"amp" + 0.067*"mehr" + 0.067*"haltet" + 0.067*"schiebt" + 0.067*"ständig" + 0.067*"kopf" + 0.067*"mio" + 0.067*"überstunden" + 0.067*"personal" + 0.067*"daher"

wirtschaft
Topic BOW: wirtschaft 
Words: 0.109*"wirtschaft" + 0.065*"leisten" + 0.065*"mindestlöhne" + 0.065*"nachbarstaaten" + 0.065*"fordern" + 0.065*"höhere" + 0.043*"transformation" + 0.043*"kretschmann" + 0.043*"ziel" + 0.043*"winfried"
Topic TF-IDF: wirtschaft 
Words: 0.066*"fordern" + 0.066*"leisten" + 0.066*"mindestlöhne" + 0.066*"nachbarstaaten" + 0.066*"höhere" + 0.051*"deutschen" + 0.051*"zahlungsmittel" + 0.051*"verhältnis" + 0.051*"digitalisierung" + 0.051*"bargeld"

bge
Topic BOW: bge 
Words: 0.083*"bge" + 0.042*"bekommt" + 0.042*"hoffe" + 0.042*"chance" + 0.042*"zwangsrentensteuer"